In [5]:
import pandas as pd

# Specify the file path
file_path = 'data_crawl.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path, encoding='utf-8')

# Display the DataFrame
print(df.shape[0])

2250


<h4>Nhóm em sử dụng file crawl data lấy từ web bán nhà đất sẽ gồm có 5 feature là giá nhà, diện tích, số phòng ngủ, số nhà vệ sinh và mô tả. Sau đó sẽ áp dụng large language model lên trên cột mô tả để lấy thêm thông tin cần chiết xuất<h4>

In [2]:

# Tính số lượng dòng mỗi file
num_rows_per_file = 2250

# Tính số lượng file cần tạo
num_files = 8

output_dir = 'C:\\Users\\Admin\\Downloads'

# Tách file thành nhiều file nhỏ
for i in range(num_files):
    start_row = i * num_rows_per_file
    end_row = (i + 1) * num_rows_per_file
    df_chunk = df.iloc[start_row:end_row]
    df_chunk.to_csv(f'{output_dir}\\file_part_{i+1}.csv', index=False)

print("Tách file CSV thành công!")


Tách file CSV thành công!


<h4>Tuy nhiên trong quá trình áp dụng LLM lên trên file gồm có 18.000 data thì thời gian chạy rất lâu lên đến 6 tiếng đồng hồ nên em quyết định sẽ tách file lớn gồm 18.000 data thành 8 file nhỏ khác nhau, sau đó sẽ sử dụng LLM lên trên mỗi file nhỏ này sau đó sẽ ghép lại để tạo được 1 file hoàn chỉnh và tiết kiệm được thời gian<h4>

In [6]:
import os
import requests
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

s = requests.Session()

api_base = "https://api.endpoints.anyscale.com/v1"
token = "esecret_51ed919nz1kq7wqv6bpgcl6pb7"
url = f"{api_base}/chat/completions"

responses = []

# Hàm để gửi yêu cầu API và xử lý phản hồi
def process_description(index, describe_text):
    text_input = f"""
    Extract information from the following information. The output is json include: House price, Area, Bedrooms, Bathrooms,Address, Length, Width, Facade, Floors.

    In which, the output for house price, area, bedrooms, bathrooms, length, width, floors should be a single numeric value.
    Facade has only two possible results: 1 (equivalent to "Yes"), 0 (equivalent to "No").
    In case house price, area, bedrooms, bathrooms, address, length, width, facade, or floors have no data, return NULL.
    The output result should be only JSON, no explanation for the API response is needed.
    The standard unit for house price is billion.
    This is the text:
    {describe_text}
    """

    body = {
      "model": "meta-llama/Meta-Llama-3-70B-Instruct",
      "messages": [
        {
          "role": "system",
          "content": "you are a data scientist"
        },
        {
          "role": "user",
          "content": text_input
        }
      ],
      "temperature": 1,
      "max_tokens": 256,
      "top_p": 1,
      "frequency_penalty": 0
    }

    try:
        with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body, timeout=30) as resp:
            try:
                output = resp.json()

                # Trích xuất nội dung JSON từ phản hồi API
                choices = output.get('choices', [])
                if choices:
                    content = choices[0].get('message', {}).get('content', '')
                    json_start = content.find('{')
                    json_end = content.rfind('}') + 1
                    json_content = content[json_start:json_end]

                    # Parse the API response as a string and extract key-value pairs
                    data = {}
                    for line in json_content.split('\n'):
                        if ':' in line:
                            key_value = line.split(':', 1)
                            if len(key_value) == 2:
                                key = key_value[0].strip()
                                value = key_value[1].strip()
                                data[key] = value

                    return index, data
                else:
                    print(f"No choices in response for: {describe_text}")

            except json.JSONDecodeError as e:
                print(f"JSON decoding error with description: {describe_text}")
                print(e)

    except (requests.exceptions.RequestException, requests.exceptions.Timeout) as e:
        print(f"HTTP request error with description: {describe_text}")
        print(e)
        return None

    return None

# Sử dụng ThreadPoolExecutor để gửi yêu cầu API đồng thời
max_workers = 18  # Số luồng tối đa
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_description, index, describe_text) for index, describe_text in df['Describe'].items()]
    
    for future in as_completed(futures):
        result = future.result()
        if result:
            responses.append(result)

# Tạo DataFrame từ danh sách các tuple (index, dictionary), xử lý trường hợp thiếu dữ liệu
if responses:
    responses.sort(key=lambda x: x[0])  # Sắp xếp theo index
    df_responses = pd.DataFrame([data for index, data in responses], index=[index for index, data in responses])
else:
    df_responses = pd.DataFrame(columns=['Giá nhà', 'Diện tích', 'Phòng ngủ', 'Phòng WC', 'Địa chỉ','Bề dài','Bề ngang','Mặt tiền','Số tầng'])

# Kết hợp dữ liệu gốc và dữ liệu mới
df_combined = df.join(df_responses)

# Hiển thị DataFrame kết hợp
print(df_combined)

# Kiểm tra và ghi log số lượng dòng đầu vào và đầu ra
print(f"Số lượng dòng ban đầu: {len(df)}")
print(f"Số lượng dòng sau khi xử lý: {len(df_responses)}")


           Price   Area Bedrooms    WC                      Address  \
0     Thỏa thuận    NaN      NaN   NaN                          NaN   
1        1.35 tỷ  15 m2     2 PN  2 WC                          NaN   
2         6.5 tỷ  55 m2      NaN   NaN         Lê Văn Sỹ, Phú Nhuận   
3      980 triệu  72 m2      NaN   NaN                          NaN   
4          13 tỷ  81 m2      NaN   NaN                          NaN   
...          ...    ...      ...   ...                          ...   
2245  Thỏa thuận  70 m2      NaN   NaN               Huỳnh Văn Nghệ   
2246      5.8 tỷ  83 m2     5 PN  5 WC                  QUẬN GÒ VẤP   
2247      4.7 tỷ  85 m2     3 PN  2 WC                       QUẬN 7   
2248        6 tỷ  84 m2      NaN   NaN           Nhà Phố Thương Mại   
2249      9.4 tỷ  90 m2      NaN   NaN  Thành phố Hồ Chí Minh   

                                               Describe "House price" "Area"  \
0     Chính chủ gửi bán căn nhà hẻm đường Bông Sao, ...          1.

<h4> Nhóm em sẽ sử dụng API Anyscale để chạy LLM trên file code nhỏ,em sẽ chiết xuất 'Giá nhà', 'Diện tích', 'Phòng ngủ', 'Phòng WC', 'Địa chỉ','Bề dài','Bề ngang','Mặt tiền','Số tầng' từ mô tả bằng LLM. Em sẽ sử dụng đa luồng để gửi API đồng thời nhằm tăng thời gian chạy của code. Output sẽ gồm data crawl gốc kết hợp với 9 feature : 'Giá nhà', 'Diện tích', 'Phòng ngủ', 'Phòng WC', 'Địa chỉ','Bề dài','Bề ngang','Mặt tiền','Số tầng' lấy được từ mô tả bằng LLM <h4>

In [7]:

# Assuming df is your DataFrame
df_first_15_columns = df_combined.iloc[:, 0:15]
print(df_first_15_columns)

           Price   Area Bedrooms    WC                      Address  \
0     Thỏa thuận    NaN      NaN   NaN                          NaN   
1        1.35 tỷ  15 m2     2 PN  2 WC                          NaN   
2         6.5 tỷ  55 m2      NaN   NaN         Lê Văn Sỹ, Phú Nhuận   
3      980 triệu  72 m2      NaN   NaN                          NaN   
4          13 tỷ  81 m2      NaN   NaN                          NaN   
...          ...    ...      ...   ...                          ...   
2245  Thỏa thuận  70 m2      NaN   NaN               Huỳnh Văn Nghệ   
2246      5.8 tỷ  83 m2     5 PN  5 WC                  QUẬN GÒ VẤP   
2247      4.7 tỷ  85 m2     3 PN  2 WC                       QUẬN 7   
2248        6 tỷ  84 m2      NaN   NaN           Nhà Phố Thương Mại   
2249      9.4 tỷ  90 m2      NaN   NaN  Thành phố Hồ Chí Minh   

                                               Describe "House price" "Area"  \
0     Chính chủ gửi bán căn nhà hẻm đường Bông Sao, ...          1.

<h4> Tuy nhiên thì dữ liệu trả về của LLM sẽ có những column bị nhiễu do trong quá trình xử lý mô tả sẽ có rất nhiều mô tả viết ở những dạng khác nhau dẫn đến
việc xử lý dẫn đến sai số, vì thế nhóm em sẽ chỉ lấy 15 cột đầu tiền của output vì 15 cột đầu tiên của output sẽ có thể bao gồm được 6 features của data crawl và 9 features lấy được trong mô tả <h4>

In [8]:
df_first_15_columns.to_csv('C:\\Users\\Admin\\Downloads\\1.csv', index=False)

<h4> Sau đấy em sẽ xuất dần dần 8 output thành 8 file kết quả <h4>

In [ ]:

# Địa chỉ của 8 file CSV
csv_files = [
    'C:\\Users\\Admin\\Downloads\\1.csv',
    'C:\\Users\\Admin\\Downloads\\2.csv',
    'C:\\Users\\Admin\\Downloads\\file_part_3_update(2).csv',
    'C:\\Users\\Admin\\Downloads\\file_part_4_update (1).csv',
    'C:\\Users\\Admin\\Downloads\\5.csv',
    'C:\\Users\\Admin\\Downloads\\6_new.csv',
    'C:\\Users\\Admin\\Downloads\\file_part_7_output.csv',
    'C:\\Users\\Admin\\Downloads\\8.csv'
]

# Đọc và nối tất cả các file CSV vào một DataFrame duy nhất
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Ghi DataFrame kết hợp ra file CSV mới
output_file = 'C:\\Users\\Admin\\Downloads\\data_crawl+llm.csv'
combined_df.to_csv(output_file, index=False)

print(f"Đã ghép thành công 8 file CSV thành file {output_file}")


<h4> Em sẽ ghép 8 file này thành 1 file hoàn chỉnh bao gồm data_crawl+llm <h4>